In [8]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

boston = load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)

df['Price'] = boston.target
X_data = df.drop('Price', axis=1)
y_data = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                   test_size=0.3,
                                                   random_state=11)

model = Pipeline([('poly' , PolynomialFeatures(degree=2, include_bias=False)),
                  ('linear', LinearRegression())])

model.fit(X_train, y_train)
pred = model.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)

print(f'MSE : {mse : .4f}, RMSE : {rmse : .4f}')
print(f'R2 score : {r2_score(y_test, pred) : .4f}')

MSE :  15.3887, RMSE :  3.9228
R2 score :  0.8316
